# Kaggle Zillow Test 1

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [10]:
properties = pd.read_csv('../data/properties_2016.csv')
train = pd.read_csv('../data/train_2016_v2.csv')

/Users/Wes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
for c in properties.columns:
    properties[c]=properties[c].fillna(1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        print(c)
        print(lbl.classes_)
        properties[c] = lbl.transform(list(properties[c].values))

train_df = train.merge(properties, how='left', on='parcelid')
x_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
x_test = properties.drop(['parcelid'], axis=1)
# shape        
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

hashottuborspa
[1]
propertycountylandusecode
['0' '010' '0100' '0101' '0102' '0103' '0104' '0105' '0106' '0108' '0109'
 '010C' '010D' '010E' '010F' '010G' '010H' '010L' '010M' '010T' '010V'
 '010X' '010Y' '0110' '0111' '0113' '0114' '0115' '0118' '011C' '0120'
 '0121' '0122' '0123' '012C' '012D' '012E' '012G' '0130' '0131' '0132'
 '0133' '0134' '0140' '0141' '014D' '01C' '01DC' '01DD' '01DE' '01DG'
 '01HC' '01HE' '01MC' '01T0' '0200' '0201' '0202' '0203' '0204' '0208'
 '0209' '020A' '020C' '020D' '020E' '020G' '020M' '020V' '020X' '0210'
 '0220' '0230' '0231' '02G0' '0300' '0301' '0303' '0304' '0305' '0308'
 '0309' '030A' '030B' '030C' '030D' '030E' '030G' '030L' '030V' '0310'
 '0320' '03TO' '0400' '0401' '0402' '0403' '0404' '0409' '040A' '040B'
 '040D' '040G' '040V' '0410' '0420' '0450' '04TO' '0600' '0700' '0701'
 '070D' '070G' '070L' '070M' '070P' '070V' '070X' '0710' '0711' '071P'
 '0720' '0730' '0741' '07M' '1' '1002' '100V' '101' '1011' '1012' '1013'
 '1014' '103' '104' '105' '1

In [ ]:
# drop out ouliers
# train_df=train_df[ train_df.logerror > -0.4 ]
# train_df=train_df[ train_df.logerror < 0.4 ]
x_train=train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

print('After removing outliers:')     
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

In [6]:
# xgboost params
xgb_params = {
    'eta': 0.03,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

In [9]:
dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

In [10]:
# cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   nfold=5,
                   num_boost_round=100,
                   early_stopping_rounds=5,
                   verbose_eval=10, 
                   show_stdv=False
                  )
num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)
pred = model.predict(dtest)
y_pred=[]

[0]	train-mae:0.0684074	test-mae:0.068418
[10]	train-mae:0.0680372	test-mae:0.0682292
[20]	train-mae:0.0678142	test-mae:0.068176
[30]	train-mae:0.067676	test-mae:0.0681738
27


In [11]:
for i,predict in enumerate(pred):
    y_pred.append(str(round(predict,10)))
y_pred=np.array(y_pred)

output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})
# set col 'ParceID' to first col
cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = output[cols]
from datetime import datetime
output.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)
output 

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.640639,0.640639,0.640639,0.640639,0.640639,0.640639
1,10759547,0.00267017,0.00267017,0.00267017,0.00267017,0.00267017,0.00267017
2,10843547,0.0668027,0.0668027,0.0668027,0.0668027,0.0668027,0.0668027
3,10859147,0.0677265,0.0677265,0.0677265,0.0677265,0.0677265,0.0677265
4,10879947,0.0175468,0.0175468,0.0175468,0.0175468,0.0175468,0.0175468
5,10898347,0.0281042,0.0281042,0.0281042,0.0281042,0.0281042,0.0281042
6,10933547,0.00805527,0.00805527,0.00805527,0.00805527,0.00805527,0.00805527
7,10940747,-0.00215096,-0.00215096,-0.00215096,-0.00215096,-0.00215096,-0.00215096
8,10954547,0.636436,0.636436,0.636436,0.636436,0.636436,0.636436
9,10976347,0.0189079,0.0189079,0.0189079,0.0189079,0.0189079,0.0189079
